# Decline Curve Analysis for Production Forecasting

## **Introduction**

Decline Curve Analysis (DCA) is a cornerstone technique in petroleum engineering for forecasting oil and gas production. By analyzing historical production data, DCA estimates future production and reserves, thereby guiding development strategies and reservoir management decisions. In this notebook, we explore three widely used decline models—exponential, harmonic, and hyperbolic—and demonstrate how to fit these models to production data. Detailed explanations are provided at each step to assist beginners and new graduate engineers in understanding both the theory and the code implementation.



## **Theory of Decline Curve Analysis**

Production from wells typically declines over time due to the natural depletion of reservoir pressure and changing reservoir dynamics. Decline Curve Analysis is an empirical method that uses mathematical models to represent this decline. The three primary decline models are:

1. **Exponential Decline**
   - **Concept:** Assumes that the production rate declines at a constant percentage rate.
   - **Mathematical Formulation:**
     $$
     q(t) = q_i e^{-d_i t}
     $$
     where:
     - \(q(t)\) is the production rate at time \(t\),
     - \(q_i\) is the initial production rate,
     - \(d_i\) is the constant decline rate.
   - **Interpretation:** This model is often applicable to wells in the early life stages or in reservoirs where the pressure decline is relatively uniform.

2. **Harmonic Decline**
   - **Concept:** Assumes that the decline rate decreases over time.
   - **Mathematical Formulation:**
     $$
     q(t) = \frac{q_i}{1 + d_i t}
     $$
     - The decline rate here is inversely proportional to time.
   - **Interpretation:** This model can better capture the slowing decline of production in some reservoirs compared to the exponential model.

3. **Hyperbolic Decline**
   - **Concept:** Offers additional flexibility by introducing a hyperbolic exponent that controls the curvature of the decline.
   - **Mathematical Formulation:**
     $$
     q(t) = \frac{q_i}{\left(1 + b\,d_i\,t\right)^{1/b}}
     $$
     where:
     - \(b\) is the hyperbolic exponent.
   - **Interpretation:** When \(b = 0\), the model converges to the exponential form. Values of \(b\) between 0 and 1 allow the model to represent various declining behaviors often observed in mature reservoirs.

   
# **Import Necessary Libraries**

```python
# Importing libraries needed for data processing, visualization, and GUI
import tkinter as tk
from tkinter import ttk, filedialog
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
```

# **Define Decline Curve Models**

```python
# Define mathematical models for decline curve analysis

def exponential_decline(t, qi, di):
    """Computes production rate at time t using exponential decline."""
    return qi * np.exp(-di * t)

def harmonic_decline(t, qi, di):
    """Computes production rate at time t using harmonic decline."""
    return qi / (1 + di * t)

def hyperbolic_decline(t, qi, di, b):
    """Computes production rate at time t using hyperbolic decline."""
    return qi / ((1 + b * di * t) ** (1 / b))
```

# **Load Production Data**

```python
# Function to load production data from user-selected file

def load_data():
    """Loads production data from CSV or Excel file and prompts the user to select time and production columns."""
    global df
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx")])
    if file_path:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path, parse_dates=[0])
        else:
            df = pd.read_excel(file_path, parse_dates=[0])
        column_selection()
```

# **Select Time and Production Columns**

```python
# Function to allow user to specify which columns represent time and production data

def column_selection():
    """Opens a dialog for the user to select the relevant columns in the dataset."""
    column_window = tk.Toplevel(root)
    column_window.title("Select Columns")
    ttk.Label(column_window, text="Select Time Column:").grid(row=0, column=0)
    time_col.set('')
    ttk.Combobox(column_window, textvariable=time_col, values=list(df.columns)).grid(row=0, column=1)
    
    ttk.Label(column_window, text="Select Production Column:").grid(row=1, column=0)
    prod_col.set('')
    ttk.Combobox(column_window, textvariable=prod_col, values=list(df.columns)).grid(row=1, column=1)
    
    ttk.Button(column_window, text="Confirm Selection", command=column_window.destroy).grid(row=2, column=0, columnspan=2)
```

# **Perform Curve Fitting**

```python
# Function to fit the decline model to historical production data

def perform_curve_fit():
    """Applies nonlinear regression to fit the chosen decline model to production data."""
    global popt
    df[time_col.get()] = pd.to_datetime(df[time_col.get()])
    df.sort_values(by=time_col.get(), inplace=True)
    df["Time_Months"] = (df[time_col.get()] - df[time_col.get()].iloc[0]).dt.days / 30.0
    t = df["Time_Months"].values
    q = df[prod_col.get()].values
    
    model = decline_model.get()
    if model == "Exponential":
        popt, _ = curve_fit(exponential_decline, t, q, p0=[q[0], 0.1], maxfev=5000)
    elif model == "Harmonic":
        popt, _ = curve_fit(harmonic_decline, t, q, p0=[q[0], 0.1], maxfev=5000)
    else:
        popt, _ = curve_fit(hyperbolic_decline, t, q, p0=[q[0], 0.1, 0.5], maxfev=5000)
```

# **Forecast Production**

```python
# Function to forecast future production using the fitted model

def forecast_production():
    """Generates future production estimates based on the fitted decline curve model."""
    years = int(entry_year.get())
    t = np.arange(0, years * 12, 1)
    model = decline_model.get()
    if model == "Exponential":
        q = exponential_decline(t, *popt)
    elif model == "Harmonic":
        q = harmonic_decline(t, *popt)
    else:
        q = hyperbolic_decline(t, *popt)
    
    ax.clear()
    ax.plot(df["Time_Months"], df[prod_col.get()], 'ro-', label='Historical Data')
    ax.plot(t, q, label=f'{model} Forecast', color='blue')
    ax.set_xlabel("Time (months)")
    ax.set_ylabel("Production Rate")
    ax.legend()
    canvas.draw()
```

# **Conclusion**

"""
Decline Curve Analysis is a crucial technique for petroleum engineers to estimate recoverable reserves and forecast production rates. Understanding the physics and mathematics behind these models allows engineers to make informed decisions in reservoir management. This tool provides an interactive way to fit decline models and analyze production trends effectively.
"""


In [ ]:
# %% [markdown]
"""
# Decline Curve Analysis for Production Forecasting

## **Introduction**

Decline Curve Analysis (DCA) is a cornerstone technique in petroleum engineering for forecasting oil and gas production. By analyzing historical production data, DCA estimates future production and reserves, thereby guiding development strategies and reservoir management decisions. In this notebook, we explore three widely used decline models—exponential, harmonic, and hyperbolic—and demonstrate how to fit these models to production data. Detailed explanations are provided at each step to assist beginners and new graduate engineers in understanding both the theory and the code implementation.
"""

# %% [markdown]
"""
## **Theory of Decline Curve Analysis**

Production from wells typically declines over time due to the natural depletion of reservoir pressure and changing reservoir dynamics. Decline Curve Analysis is an empirical method that uses mathematical models to represent this decline. The three primary decline models are:

1. **Exponential Decline**
   - **Concept:** Assumes that the production rate declines at a constant percentage rate.
   - **Mathematical Formulation:**
     $$
     q(t) = q_i e^{-d_i t}
     $$
     where:
     - \(q(t)\) is the production rate at time \(t\),
     - \(q_i\) is the initial production rate,
     - \(d_i\) is the constant decline rate.
   - **Interpretation:** This model is often applicable to wells in the early life stages or in reservoirs where the pressure decline is relatively uniform.

2. **Harmonic Decline**
   - **Concept:** Assumes that the decline rate decreases over time.
   - **Mathematical Formulation:**
     $$
     q(t) = \frac{q_i}{1 + d_i t}
     $$
     - The decline rate here is inversely proportional to time.
   - **Interpretation:** This model can better capture the slowing decline of production in some reservoirs compared to the exponential model.

3. **Hyperbolic Decline**
   - **Concept:** Offers additional flexibility by introducing a hyperbolic exponent that controls the curvature of the decline.
   - **Mathematical Formulation:**
     $$
     q(t) = \frac{q_i}{\left(1 + b\,d_i\,t\right)^{1/b}}
     $$
     where:
     - \(b\) is the hyperbolic exponent.
   - **Interpretation:** When \(b = 0\), the model converges to the exponential form. Values of \(b\) between 0 and 1 allow the model to represent various declining behaviors often observed in mature reservoirs.
"""

# %% 
# Import necessary libraries for GUI creation, plotting, and data analysis
import tkinter as tk                    # For creating a simple GUI interface
from tkinter import ttk, filedialog     # For advanced widgets and file dialogs
import matplotlib.pyplot as plt         # For data visualization
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg  # To integrate matplotlib plots into Tkinter
import numpy as np                      # For numerical operations and handling arrays
import pandas as pd                     # For data manipulation and analysis
from scipy.optimize import curve_fit    # For performing non-linear regression on our decline models

# Global variables for use across functions
df = None
popt = None
time_col = tk.StringVar()
prod_col = tk.StringVar()
decline_model = tk.StringVar()

# %% [markdown]
"""
## **Step 2: Define Decline Curve Models**

The following functions represent the three decline models:
- **Exponential Decline:** Assumes production rate declines exponentially.
- **Harmonic Decline:** Models a declining rate that is inversely proportional to time.
- **Hyperbolic Decline:** Incorporates a hyperbolic exponent to add flexibility in modeling decline behavior.
"""

# %%
# Define mathematical models for decline curve analysis
```python
def exponential_decline(t, qi, di):
    """
    Computes production rate at time t using exponential decline.
    
    Parameters:
        t  : Time variable (in months or years)
        qi : Initial production rate
        di : Constant exponential decline rate
        
    Returns:
        q(t) : Production rate at time t
    """
    return qi * np.exp(-di * t)

def harmonic_decline(t, qi, di):
    """
    Computes production rate at time t using harmonic decline.
    
    Parameters:
        t  : Time variable (in months or years)
        qi : Initial production rate
        di : Decline rate coefficient
        
    Returns:
        q(t) : Production rate at time t
    """
    return qi / (1 + di * t)

def hyperbolic_decline(t, qi, di, b):
    """
    Computes production rate at time t using hyperbolic decline.
    
    Parameters:
        t  : Time variable (in months or years)
        qi : Initial production rate
        di : Decline rate coefficient
        b  : Hyperbolic exponent controlling curvature
        
    Returns:
        q(t) : Production rate at time t
    """
    return qi / ((1 + b * di * t) ** (1 / b))

# %% [markdown]
"""
## **Step 3: Load Production Data**

This function loads production data from a CSV or Excel file. The first column is assumed to be a date, which is parsed automatically. After loading the data, a column selection function is called to allow the user to specify which columns represent time and production data.
"""

# %%
# Function to load production data from a user-selected file

def load_data():
    """
    Loads production data from CSV or Excel file.
    Opens a file dialog for the user to select a file.
    The file is read into a pandas DataFrame, and the first column is parsed as dates.
    
    Once the file is loaded, the column_selection function is called
    to allow the user to choose the time and production columns.
    """
    global df
    file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv"), ("Excel files", "*.xlsx")])
    if file_path:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path, parse_dates=[0])
        else:
            df = pd.read_excel(file_path, parse_dates=[0])
        column_selection()

# %% [markdown]
"""
## **Step 4: Select Time and Production Columns**

This function opens a new window that allows users to select the appropriate columns for time and production from the loaded dataset.
"""

# %%
# Function to allow the user to specify which columns represent time and production data

def column_selection():
    """
    Opens a dialog for the user to select the relevant columns in the dataset.
    This function creates a new window with dropdown menus populated with the DataFrame's column names.
    Users must select one column for time and another for production.
    """
    column_window = tk.Toplevel(root)
    column_window.title("Select Columns")
    
    ttk.Label(column_window, text="Select Time Column:").grid(row=0, column=0)
    time_col.set('')  # Reset the time column variable
    ttk.Combobox(column_window, textvariable=time_col, values=list(df.columns)).grid(row=0, column=1)
    
    ttk.Label(column_window, text="Select Production Column:").grid(row=1, column=0)
    prod_col.set('')  # Reset the production column variable
    ttk.Combobox(column_window, textvariable=prod_col, values=list(df.columns)).grid(row=1, column=1)
    
    ttk.Button(column_window, text="Confirm Selection", command=column_window.destroy).grid(row=2, column=0, columnspan=2)

# %% [markdown]
"""
## **Step 5: Perform Curve Fitting**

This function applies non-linear regression to fit the chosen decline model to the historical production data. The steps include:
1. Converting the time column to datetime and sorting the data.
2. Calculating the elapsed time in months.
3. Extracting time and production data as numpy arrays.
4. Fitting the selected model (Exponential, Harmonic, or Hyperbolic) using the `curve_fit` function.
The optimized parameters are stored in the global variable `popt` for forecasting.
"""

# %%
# Function to fit the decline model to historical production data

def perform_curve_fit():
    """
    Applies nonlinear regression to fit the chosen decline model to the production data.
    
    Steps:
    1. Convert the selected time column to datetime.
    2. Sort the DataFrame by time.
    3. Compute the time difference in months from the start date.
    4. Extract time and production data into numpy arrays.
    5. Select the decline model based on user input (Exponential, Harmonic, or Hyperbolic).
    6. Use curve_fit from scipy.optimize to estimate the model parameters.
    
    The estimated parameters are stored in the global variable 'popt' for later forecasting.
    """
    global popt
    df[time_col.get()] = pd.to_datetime(df[time_col.get()])
    df.sort_values(by=time_col.get(), inplace=True)
    # Calculate time in months from the initial production date
    df["Time_Months"] = (df[time_col.get()] - df[time_col.get()].iloc[0]).dt.days / 30.0
    t = df["Time_Months"].values
    q = df[prod_col.get()].values
    
    model = decline_model.get()
    if model == "Exponential":
        popt, _ = curve_fit(exponential_decline, t, q, p0=[q[0], 0.1], maxfev=5000)
    elif model == "Harmonic":
        popt, _ = curve_fit(harmonic_decline, t, q, p0=[q[0], 0.1], maxfev=5000)
    else:  # Hyperbolic
        popt, _ = curve_fit(hyperbolic_decline, t, q, p0=[q[0], 0.1, 0.5], maxfev=5000)

# %% [markdown]
"""
## **Step 6: Forecast Production**

This function uses the optimized model parameters to forecast future production. The process involves:
1. Reading the forecast period (in years) from user input.
2. Creating a time array for the forecast period (in months).
3. Calculating the forecasted production using the selected decline model.
4. Plotting both historical data and forecasted production on the same graph for visual comparison.
"""

# %%
# Function to forecast future production using the fitted decline curve model

def forecast_production():
    """
    Generates future production estimates based on the fitted decline curve model.
    
    Steps:
    1. Retrieve the number of years for the forecast from user input.
    2. Create a time array (in months) covering the forecast period.
    3. Use the fitted model parameters (stored in 'popt') to compute production rates over time.
    4. Clear the current plot and display both historical data and the forecast curve.
    
    The resulting plot provides a visual comparison between historical production data and the model forecast.
    """
    years = int(entry_year.get())
    # Create a time array in months for the forecast period
    t = np.arange(0, years * 12, 1)
    model = decline_model.get()
    if model == "Exponential":
        q = exponential_decline(t, *popt)
    elif model == "Harmonic":
        q = harmonic_decline(t, *popt)
    else:  # Hyperbolic
        q = hyperbolic_decline(t, *popt)
    
    # Update the plot with historical data and forecast
    ax.clear()
    ax.plot(df["Time_Months"], df[prod_col.get()], 'ro-', label='Historical Data')
    ax.plot(t, q, label=f'{model} Forecast', color='blue')
    ax.set_xlabel("Time (months)")
    ax.set_ylabel("Production Rate")
    ax.legend()
    canvas.draw()

# %% [markdown]
"""
## **Conclusion**

Decline Curve Analysis is a vital tool in petroleum engineering used to estimate recoverable reserves and forecast future production rates. This notebook demonstrates the implementation of exponential, harmonic, and hyperbolic decline models. Through detailed explanations and step-by-step code walkthroughs, new graduates and early career petroleum engineers can gain both a theoretical understanding and practical insights into applying DCA for reservoir management.
"""

# %% 
# Set up the main Tkinter GUI window and matplotlib canvas

root = tk.Tk()
root.title("Decline Curve Analysis")

# Create a matplotlib figure and axes
fig, ax = plt.subplots(figsize=(8, 5))
canvas = FigureCanvasTkAgg(fig, master=root)
canvas_widget = canvas.get_tk_widget()
canvas_widget.pack(side=tk.TOP, fill=tk.BOTH, expand=1)

# Create a controls frame for file loading, model selection, and forecasting input
controls_frame = ttk.Frame(root)
controls_frame.pack(side=tk.BOTTOM, fill=tk.X)

# File Load Button
ttk.Button(controls_frame, text="Load Data", command=load_data).pack(side=tk.LEFT, padx=5, pady=5)

# Decline Model Selection Dropdown
decline_model.set("Exponential")  # Default selection
ttk.Label(controls_frame, text="Select Decline Model:").pack(side=tk.LEFT, padx=5)
model_dropdown = ttk.Combobox(controls_frame, textvariable=decline_model, values=["Exponential", "Harmonic", "Hyperbolic"])
model_dropdown.pack(side=tk.LEFT, padx=5)

# Curve Fit Button
ttk.Button(controls_frame, text="Perform Curve Fit", command=perform_curve_fit).pack(side=tk.LEFT, padx=5)

# Forecast Input and Button
ttk.Label(controls_frame, text="Forecast Years:").pack(side=tk.LEFT, padx=5)
entry_year = ttk.Entry(controls_frame, width=5)
entry_year.pack(side=tk.LEFT, padx=5)
ttk.Button(controls_frame, text="Forecast Production", command=forecast_production).pack(side=tk.LEFT, padx=5)

# Start the Tkinter main loop
root.mainloop()
